In [75]:
import numpy as np 
import yfinance as yf
import matplotlib.pyplot as plt 
import pandas as pd 
#import backtesting as bt
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas_ta as ta
# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG

stock_code = "GE" 
data_collecting_begaining_date = "2020-01-01"
data_collecting_ending_date = "2024-01-01"
period_of_collecting = "1d" 

def stock_price_collecting(code, datestart, dateend, period):
    data = yf.Ticker(code)
    prices = data.history(start=datestart, end=dateend, interval=period)
    return prices

historical_price_data = stock_price_collecting(stock_code, data_collecting_begaining_date, data_collecting_ending_date, period_of_collecting)
historical_price_data = pd.DataFrame(historical_price_data)

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

def BOLL(values, n, m):
    """
    Return Bollinger Bands for `values`, at
    each step taking into account `n` previous values
    and `m` standard deviations away from the mean.
    """
    sma = SMA(values, n)
    std = pd.Series(values).rolling(n).std()
    return sma, sma + m * std, sma - m * std

def Dochian(data, high_length, low_length):
    """
    Return Dochian Channel for `values`, at
    each step taking into account `n` previous values
    """
    return ta.donchian(data.High.s, data.Low.s, low_length, high_length, offset=1, talib=False)

def ATR(data, n):
    return np.array(ta.atr(high=data.High.s, low=data.Low.s, close=data.Close.s, length=n, talib=False))
    
    # tr = pd.Series(np.maximum(high - low, high - close, close - low))
    # return tr.rolling(n).mean()

class turtleTrading(Strategy):
    
    def init(self):    
        #print(ATR(self.data, 20))
        self.atr = self.I(ATR, self.data, 60)
        self.donchian = self.I(Dochian, self.data, 60,50)
        self.last_buy_price = 0
        self.last_buy_atr = 0
        
    def next(self):
        # If today's close exceed the donchian_upper_bound, buy, record price and ATR
        if crossover(self.data.Close, self.donchian[2]):
            portion = min((0.01*self.data.Close[-1])/self.atr[-1], 1)
            self.buy(size=portion)
            self.last_buy_price = self.data.Close[-1]
            self.last_buy_atr = self.atr[-1]
        # If today's close is below the donchian_lower_bound, sell (end trade)
        elif crossover(self.donchian[0], self.data.Close):
            self.position.close() 
        # If price decresed by twice N at the last buy point, sell all (stop loss)
        elif crossover(self.last_buy_price - 2.0*self.last_buy_atr, self.data.Close[-1]):
            self.position.close()

bt = Backtest(historical_price_data, turtleTrading, cash=1_000_000)
stats = bt.run()
print(stats)
bt.plot()

Start                     2020-01-02 00:00...
End                       2023-12-29 00:00...
Duration                   1457 days 00:00:00
Exposure Time [%]                   49.900596
Equity Final [$]                1865867.65195
Equity Peak [$]                1910227.622729
Return [%]                          86.586765
Buy & Hold Return [%]               73.985317
Return (Ann.) [%]                   16.910856
Volatility (Ann.) [%]               21.236347
Sharpe Ratio                         0.796317
Sortino Ratio                        1.483993
Calmar Ratio                         0.475718
Max. Drawdown [%]                  -35.548036
Avg. Drawdown [%]                   -4.329359
Max. Drawdown Duration      691 days 00:00:00
Avg. Drawdown Duration       46 days 00:00:00
# Trades                                   36
Win Rate [%]                        69.444444
Best Trade [%]                      66.122704
Worst Trade [%]                     -21.66159
Avg. Trade [%]                    

/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p27371', ...)